In [1]:
%load_ext autoreload
import os
import sys

In [2]:
os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ["OPENAI_API_KEY"]= 'xxx'
os.environ['CUDA_VISIBLE_DEVICES'] = '4'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
sys.path.append('./src-py')

In [3]:
import json
import os
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import re
from collections import Counter

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


%autoreload
import utils
import prompts
import random

from tabulate import tabulate
import tiktoken
from llm_based_evaluation import *

/mnt/swordfish-pool2/milad/conda-envs/datadreamer/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
from transformers import AutoTokenizer, pipeline

In [5]:
import json
keys = json.load(open('./keys.json'))
for key, val in keys.items():
    os.environ[key] = val

In [6]:
from huggingface_hub import login

login(os.environ['hf_token'])

In [7]:
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/'
models_folder = "/mnt/swordfish-pool2/milad/communicating-science-to-the-public/models/"

In [8]:
gpt_tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

### Evaluate Science Models

- Now we will evalaute the following models on a sample from the test set using only the generic prompt
    - LLAMA-3 baseline
    - Qwen baseline
    - LLAMA-3 fine-tuned on DeepSeek generated conversations
    - Qwen  fine-tuned on DeepSeek generated conversations

### Evaluation

#### Basic Evaluation

In [9]:
ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public'

In [24]:
generated_convs = {
    #'gpt3-baseline' : datasets.load_from_disk(ds_path + '/gpt3-test-conv-ds'),
    #'llama3-baseline' : datasets.load_from_disk(ds_path + '/baseline-llama3-test-conv-ds'),
    'llama3-baseline-adv-prompt':datasets.load_from_disk(ds_path + '/baseline-advanced-prompt-llama3-test-conv-ds/'),
    #'ft-40k-llama3-on-deepseek' :datasets.load_from_disk(ds_path + '/ft-40k-llama3-test-conv-ds'),
    'ft-llama3-on-deepseek' :datasets.load_from_disk(ds_path + '/ft-llama3-test-conv-ds'),
    'qwen-baseline-adv-prompt':datasets.load_from_disk(ds_path + '/baseline-qwen-test-conv-ds'),
    #'ft-qwen-on-deepseek' :datasets.load_from_disk(ds_path + '/ft-40k-qwen-test-conv-ds/'),
    'ft-qwen-on-deepseek' :datasets.load_from_disk(ds_path + '/ft-qwen-full-ds-training-test-conv-ds'),
}

In [25]:
# generated_convs['ft-llama3-on-deepseek'] = generated_convs['ft-llama3-on-deepseek'].map(lambda row: {'conversation': '\n\n'.join(['{}: {}'.format('Journalist', x['content']) if x['role'] == 'assistant' else '{}: {}'.format('Researcher', x['content']) for x in row['generated_conversation'][1:]])})
# generated_convs['llama3-baseline'] = generated_convs['llama3-baseline'].map(lambda row: {'conversation': '\n\n'.join(['{}: {}'.format('Journalist', x['content']) if x['role'] == 'assistant' else '{}: {}'.format('Researcher', x['content']) for x in row['generated_conversation'][1:]])})

In [26]:
#generated_convs['ft-40k-llama3-on-deepseek'] = generated_convs['ft-40k-llama3-on-deepseek'].map(lambda row: {'pr-article': paper_id_to_article[row['paper_id']]})
generated_convs['ft-llama3-on-deepseek'] = generated_convs['ft-llama3-on-deepseek'].map(lambda row: {'pr-article': paper_id_to_article[row['paper_id']]})
generated_convs['llama3-baseline-adv-prompt'] = generated_convs['llama3-baseline-adv-prompt'].map(lambda row: {'pr-article': paper_id_to_article[row['paper_id']]})
generated_convs['qwen-baseline-adv-prompt'] = generated_convs['qwen-baseline-adv-prompt'].map(lambda row: {'pr-article': paper_id_to_article[row['paper_id']]})
generated_convs['ft-qwen-on-deepseek'] = generated_convs['ft-qwen-on-deepseek'].map(lambda row: {'pr-article': paper_id_to_article[row['paper_id']]})

In [27]:
def evalaute_convs(datasets):
    eval_results = {}
    for name, ds in datasets.items():
        eval_results[name] = utils.evaluate_conv(ds['conversation'], None, ds['pr-article'])

    print(tabulate(
        [[name] + list(eval_res.values())[:3] for name, eval_res in eval_results.items()],
        headers=['Prompt', 'Rouge-1', 'Rouge-L', 'BERT-f1']
    ))

In [28]:
all_synth_conversations = evalaute_convs(generated_convs)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prompt                        Rouge-1    Rouge-L    BERT-f1
--------------------------  ---------  ---------  ---------
llama3-baseline-adv-prompt      0.337      0.145      0.82
ft-llama3-on-deepseek           0.346      0.148      0.821
qwen-baseline-adv-prompt        0.3        0.127      0.824
ft-qwen-on-deepseek             0.28       0.125      0.808


In [136]:
all_synth_conversations = evalaute_convs(generated_convs)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prompt                        Rouge-1    Rouge-L    BERT-f1
--------------------------  ---------  ---------  ---------
llama3-baseline                 0.333      0.146      0.819
llama3-baseline-adv-prompt      0.337      0.145      0.82
ft-40k-llama3-on-deepseek       0.425      0.162      0.83
qwen-baseline-adv-prompt        0.3        0.127      0.824
ft-qwen-on-deepseek             0.397      0.152      0.83


#### LLM-based Evaluation

In [10]:
ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public'
generated_convs = {
    'llama3-baseline-adv-prompt':[datasets.load_from_disk(ds_path + '/baseline-advanced-prompt-llama3-test-conv-ds/'), ds_path + '/baseline-advanced-prompt-llama3-test-conv-ds/'],
    'qwen-baseline-adv-prompt':[datasets.load_from_disk(ds_path + '/baseline-qwen-test-conv-ds/'), ds_path + '/baseline-qwen-test-conv-ds/'],
    #'ft-40k-llama3-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft-40k-llama3-test-conv-ds/'),ds_path +  '/ft-40k-llama3-test-conv-ds/'],
    #'ft-40k-qwen-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft-40k-qwen-test-conv-ds/'),ds_path +  '/ft-40k-qwen-test-conv-ds/'],
    'ft-llama3-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft-llama3-test-conv-ds/'),ds_path +  '/ft-llama3-test-conv-ds/'],
    'ft-qwen-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft-qwen-full-ds-training-test-conv-ds/'),ds_path +  '/ft-qwen-full-ds-training-test-conv-ds'],
}

In [20]:
# generated_convs['ft-llama3-on-deepseek'][0] = generated_convs['ft-llama3-on-deepseek'][0].map(lambda row: {'conversation': '\n\n'.join(['{}: {}'.format('Journalist', x['content']) if x['role'] == 'assistant' else '{}: {}'.format('Researcher', x['content']) for x in row['generated_conversation'][1:]])})
# generated_convs['llama3-baseline'][0] = generated_convs['llama3-baseline'][0].map(lambda row: {'conversation': '\n\n'.join(['{}: {}'.format('Journalist', x['content']) if x['role'] == 'assistant' else '{}: {}'.format('Researcher', x['content']) for x in row['generated_conversation'][1:]])})
# generated_convs['gpt3-baseline'][0] = generated_convs['gpt3-baseline'][0].remove_columns(['societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed'])
# generated_convs['llama3-baseline'][0] = generated_convs['llama3-baseline'][0].remove_columns(['societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed'])
# generated_convs['ft-llama3-on-deepseek'][0] = generated_convs['ft-llama3-on-deepseek'][0].remove_columns(['societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed'])

In [138]:
prompts_to_eval = [prompts.clarity_eval_prompt, prompts.scientific_context_eval_prompt, prompts.societal_context_eval_prompt]

llm_eval_results = llm_based_evaluation(prompts_to_eval, generated_convs, force_generation=True)

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [42:30<00:00,  5.10s/it]


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [42:26<00:00,  5.09s/it]


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [36:29<00:00,  4.38s/it]


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [35:34<00:00,  4.27s/it]


Saving the dataset (0/1 shards):   0%|          | 0/500 [00:00<?, ? examples/s]

#                             clarity_eval_prompt    scientific_eval_prompt    societal_eval_prompt    Avg
--------------------------  ---------------------  ------------------------  ----------------------  -----
llama3-baseline-adv-prompt                   4.19                      2.17                    1.58   2.65
qwen-baseline-adv-prompt                     4.18                      2.41                    1.85   2.81
ft-40k-llama3-on-deepseek                    4.48                      1.9                     1.75   2.71
ft-40k-qwen-on-deepseek                      4.43                      1.94                    1.67   2.68


In [21]:
prompts_to_eval = [prompts.clarity_eval_prompt, prompts.scientific_context_eval_prompt, prompts.societal_context_eval_prompt]

llm_eval_results = llm_based_evaluation(prompts_to_eval, generated_convs, force_generation=True)

Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/baseline-advanced-prompt-llama3-test-conv-ds/ from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/baseline-qwen-test-conv-ds/ from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/ft-40k-llama3-test-conv-ds/ from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/ft-40k-qwen-test-conv-ds/ from already saved file
#                             clarity_eval_prompt    scientific_eval_prompt    societal_eval_prompt    Avg
--------------------------  ---------------------  ------------------------  ----------------------  -----
llama3-baseline-adv-prompt                   4.07                      2.3                     1.46   2.61
qwen-baseline-adv-prompt                     3.91                      2.39                    1.63   2.64
ft-40k-llama3-on-deepseek                    4.52               

-------

#### Evaluating the ground-truth generated convs:

In [13]:
gt_convs = {'original_deepseek_conv': [datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/processed_test_ds_sample'), '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/processed_test_ds_sample']}

In [14]:
prompts_to_eval = [prompts.clarity_eval_prompt, prompts.scientific_context_eval_prompt, prompts.societal_context_eval_prompt]

llm_eval_gt_results = llm_based_evaluation(prompts_to_eval, gt_convs, force_generation=False)

Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/processed_test_ds_sample from already saved file
#                         clarity_eval_prompt    scientific_eval_prompt    societal_eval_prompt    Avg
----------------------  ---------------------  ------------------------  ----------------------  -----
original_deepseek_conv                   4.52                       2.2                    2.06   2.93


------

### Analysis of the automatic evalaution

In [21]:
# To add the topic to the data
deepseek_conv_dataset = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-cleaned/')
paper_id_to_article = {x['id']: x['pr-article'] for x in deepseek_conv_dataset}
paper_id_to_topic = {x['id']: x['Topic'] for x in deepseek_conv_dataset}

In [19]:
ds_path = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public'
generated_convs = {
    'llama3-baseline-adv-prompt':[datasets.load_from_disk(ds_path + '/baseline-advanced-prompt-llama3-test-conv-ds/'), ds_path + '/baseline-advanced-prompt-llama3-test-conv-ds/'],
    'qwen-baseline-adv-prompt':[datasets.load_from_disk(ds_path + '/baseline-qwen-test-conv-ds/'), ds_path + '/baseline-qwen-test-conv-ds/'],
    'ft-llama3-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft-llama3-test-conv-ds/'),ds_path +  '/ft-llama3-test-conv-ds/'],
    'ft-qwen-on-deepseek' :[datasets.load_from_disk(ds_path + '/ft-40k-qwen-test-conv-ds/'),ds_path +  '/ft-40k-qwen-test-conv-ds/'],
}

In [27]:
generated_convs['llama3-baseline-adv-prompt'][0] = generated_convs['llama3-baseline-adv-prompt'][0].map(lambda row: {'topic': paper_id_to_topic[row['paper_id']]})
generated_convs['qwen-baseline-adv-prompt'][0] = generated_convs['qwen-baseline-adv-prompt'][0].map(lambda row: {'topic': paper_id_to_topic[row['paper_id']]})
generated_convs['ft-llama3-on-deepseek'][0] = generated_convs['ft-llama3-on-deepseek'][0].map(lambda row: {'topic': paper_id_to_topic[row['paper_id']]})
generated_convs['ft-qwen-on-deepseek'][0] = generated_convs['ft-qwen-on-deepseek'][0].map(lambda row: {'topic': paper_id_to_topic[row['paper_id']]})

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [20]:
prompts_to_eval = [prompts.clarity_eval_prompt, prompts.scientific_context_eval_prompt, prompts.societal_context_eval_prompt]

llm_eval_results = llm_based_evaluation(prompts_to_eval, generated_convs, force_generation=False)

Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/baseline-advanced-prompt-llama3-test-conv-ds/ from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/baseline-qwen-test-conv-ds/ from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/ft-llama3-test-conv-ds/ from already saved file
Loading /mnt/swordfish-pool2/milad/communicating-science-to-the-public/ft-40k-qwen-test-conv-ds/ from already saved file
#                             clarity_eval_prompt    scientific_eval_prompt    societal_eval_prompt    Avg
--------------------------  ---------------------  ------------------------  ----------------------  -----
llama3-baseline-adv-prompt                   4.19                      2.17                    1.58   2.65
qwen-baseline-adv-prompt                     4.18                      2.41                    1.85   2.81
ft-llama3-on-deepseek                        4.2                    

In [33]:
print(tabulate(
        [[name] + get_llm_avg_scores(res, prompts_to_eval) for name, res in llm_eval_results.items()],
        headers=['#'] + [p['strategy_name'] for p in prompts_to_eval] + ['Avg']
    ))

#                             clarity_eval_prompt    scientific_eval_prompt    societal_eval_prompt    Avg
--------------------------  ---------------------  ------------------------  ----------------------  -----
llama3-baseline-adv-prompt                   4.19                      2.17                    1.58   2.65
qwen-baseline-adv-prompt                     4.18                      2.41                    1.85   2.81
ft-llama3-on-deepseek                        4.2                       2.04                    1.74   2.66
ft-qwen-on-deepseek                          4.43                      1.94                    1.67   2.68


In [34]:
for topic in set(paper_id_to_topic.values()):
    print(topic)
    print(tabulate(
        [[name] + get_llm_avg_scores(res.filter(lambda row: row['topic'] ==topic), prompts_to_eval) for name, res in llm_eval_results.items()],
        headers=['#'] + [p['strategy_name'] for p in prompts_to_eval] + ['Avg']
    ))

Space


Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

KeyError: 'topic'